<a href="https://colab.research.google.com/github/jakeliu15/rlmarketpredictor/blob/main/Jacob_Liu_Clean_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and Import Packages

In [ ]:
## install required packages
!pip install swig
!pip install wrds
!pip install pyportfolioopt
## install finrl library
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 12.0 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
  Attempting uninstall: packaging
    Found existing installation: packaging 24.1
    Uninstalling packaging-24.1:
      Successfully uninstalled packaging-24.1
  Attempting uninstall: pandas
    Found existing installation: pandas 2.1.4
    Uninstalling pandas-2.1.4:
      Successfully uninstalled

In [ ]:
import pandas as pd
import numpy as np
import datetime
import yfinance as yf

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl import config_tickers
from finrl.config import INDICATORS

import itertools

In [ ]:
TRAIN_START_DATE = '2009-01-01'
TRAIN_END_DATE = '2020-07-01'
TRADE_START_DATE = '2020-07-01'
TRADE_END_DATE = '2023-05-01'

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
symbols = [
    'aapl',
    'msft',
    'meta',
    'ibm',
    'hd',
    'cat',
    'amzn',
    'intc',
    't',
    'v'
    'gs'
]

In [ ]:
df_raw = YahooDownloader(start_date = TRAIN_START_DATE,
                                end_date = TRADE_END_DATE,
                                ticker_list = symbols).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Shape of DataFrame:  (32958, 8)


In [ ]:
df_raw.head()

,date,open,high,low,close,volume,tic,day
0,2009-01-02,3.067143,3.251429,3.041429,2.737004,746015200,aapl,4
1,2009-01-02,2.567500,2.726500,2.553500,2.718000,145928000,amzn,4
2,2009-01-02,44.910000,46.980000,44.709999,30.712526,7117200,cat,4
3,2009-01-02,23.070000,24.190001,22.959999,16.336679,14902500,hd,4
4,2009-01-02,80.200768,83.738052,80.200768,48.733894,7905877,ibm,4


# Preprocess Data

In [ ]:
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_vix=True,
                     use_turbulence=True,
                     user_defined_feature = False)

processed = fe.preprocess_data(df_raw)

Successfully added technical indicators


[*********************100%***********************]  1 of 1 completed


Shape of DataFrame:  (3604, 8)
Successfully added vix
Successfully added turbulence index


In [ ]:
processed

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2009-01-02,3.067143,3.251429,3.041429,2.737004,746015200,aapl,4,0.000000,2.958118,2.631402,100.000000,66.666667,100.000000,2.737004,2.737004,39.189999,0.000000
1,2009-01-02,2.567500,2.726500,2.553500,2.718000,145928000,amzn,4,0.000000,2.958118,2.631402,100.000000,66.666667,100.000000,2.718000,2.718000,39.189999,0.000000
2,2009-01-02,44.910000,46.980000,44.709999,30.712526,7117200,cat,4,0.000000,2.958118,2.631402,100.000000,66.666667,100.000000,30.712526,30.712526,39.189999,0.000000
3,2009-01-02,23.070000,24.190001,22.959999,16.336679,14902500,hd,4,0.000000,2.958118,2.631402,100.000000,66.666667,100.000000,16.336679,16.336679,39.189999,0.000000
4,2009-01-02,80.200768,83.738052,80.200768,48.733894,7905877,ibm,4,0.000000,2.958118,2.631402,100.000000,66.666667,100.000000,48.733894,48.733894,39.189999,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28827,2023-04-27,288.000000,294.709991,287.799988,282.637177,3133600,hd,3,0.850852,291.156088,274.253897,48.591915,15.266338,7.315064,279.781629,286.247869,17.030001,5.475523
28828,2023-04-27,126.370003,127.019997,125.459999,119.107483,3204900,ibm,3,-0.770918,124.905762,116.251010,44.911363,-38.511439,1.220107,119.846374,121.489782,17.030001,5.475523
28829,2023-04-27,28.750000,29.950001,28.510000,29.144178,60186200,intc,3,-0.018558,33.393176,28.197082,50.727391,-66.435398,13.482737,30.099059,28.330523,17.030001,5.475523
28830,2023-04-27,295.970001,305.200012,295.250000,301.261169,46462600,msft,3,5.080237,295.548257,272.647746,63.072915,228.756228,42.136850,280.462566,267.492239,17.030001,5.475523


In [ ]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
processed_full

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2009-01-02,aapl,3.067143,3.251429,3.041429,2.737004,746015200.0,4.0,0.000000,2.958120,2.631401,100.000000,66.666667,100.000000,2.737004,2.737004,39.189999,0.000000
1,2009-01-02,amzn,2.567500,2.726500,2.553500,2.718000,145928000.0,4.0,0.000000,2.958120,2.631401,100.000000,66.666667,100.000000,2.718000,2.718000,39.189999,0.000000
2,2009-01-02,cat,44.910000,46.980000,44.709999,30.712511,7117200.0,4.0,0.000000,2.958120,2.631401,100.000000,66.666667,100.000000,30.712511,30.712511,39.189999,0.000000
3,2009-01-02,hd,23.070000,24.190001,22.959999,16.336674,14902500.0,4.0,0.000000,2.958120,2.631401,100.000000,66.666667,100.000000,16.336674,16.336674,39.189999,0.000000
4,2009-01-02,ibm,80.200768,83.738052,80.200768,48.733883,7905877.0,4.0,0.000000,2.958120,2.631401,100.000000,66.666667,100.000000,48.733883,48.733883,39.189999,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41827,2023-04-27,hd,288.000000,294.709991,287.799988,282.637177,3133600.0,3.0,0.850854,291.156059,274.253920,48.591913,15.266367,7.315064,279.781625,286.247866,17.030001,5.475583
41828,2023-04-27,ibm,126.370003,127.019997,125.459999,119.107491,3204900.0,3.0,-0.770917,124.905767,116.251008,44.911381,-38.511381,1.220107,119.846376,121.489782,17.030001,5.475583
41829,2023-04-27,intc,28.750000,29.950001,28.510000,29.144178,60186200.0,3.0,-0.018558,33.393176,28.197081,50.727391,-66.435396,13.482737,30.099059,28.330523,17.030001,5.475583
41830,2023-04-27,msft,295.970001,305.200012,295.250000,301.261139,46462600.0,3.0,5.080235,295.548231,272.647737,63.072936,228.756221,42.136850,280.462552,267.492234,17.030001,5.475583


# Save the Data

In [ ]:
# Split the data
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

23144
5688


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

train_path = '/content/drive/MyDrive/rlmarketpredictor/train.csv'
trade_path = '/content/drive/MyDrive/rlmarketpredictor/trade.csv'

with open(train_path, 'w', encoding = 'utf-8-sig') as f:
  train.to_csv(f)

with open(trade_path, 'w', encoding = 'utf-8-sig') as f:
  trade.to_csv(f)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
